In [4]:
import os
import pandas as pd

# 1) Create output folder
os.makedirs("cf_dataset_ml", exist_ok=True)

# 2) Load your 90k handles file
HANDLES_PATH = "/Users/mohamedasifa/Desktop/SheetGeneration/codeforces_ALL_USERS_COMPLETE.csv"   # <-- change this file name/path
COL = "Username"

df = pd.read_csv(HANDLES_PATH)

# 3) Clean handles
handles = (
    df[COL]
    .dropna()
    .astype(str)
    .str.strip()
)
handles = handles[handles != ""].drop_duplicates()

print("Total unique handles available:", len(handles))

# 4) Sample random 3000 (reproducible)
users_3k = handles.sample(n=3000, random_state=42).tolist()

# 5) Save
out_path = "cf_dataset_ml/users_3k.csv"
pd.Series(users_3k, name="user_id").to_csv(out_path, index=False)

print("✅ Saved:", out_path, "| Count:", len(users_3k))
print("Example handles:", users_3k[:10])


/var/folders/dn/6pycpdf50h17xp8s7qg7y93m0000gn/T/ipykernel_26359/2502829658.py:11: DtypeWarning: Columns (0: openId) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(HANDLES_PATH)


Total unique handles available: 898802
✅ Saved: cf_dataset_ml/users_3k.csv | Count: 3000
Example handles: ['Ravinder2000', 'sivapavan1998', 'noelkawn', 'hoangle10tin', 'R_K_RATHORE', 'manhvinh', 'dhruvsavani008', '_ud', 'Vibhum', 'Sarfarozkhuja']


In [5]:
import os
import time
import json
import requests
import pandas as pd
from tqdm import tqdm

# -----------------------------
# Config
# -----------------------------
BASE = "https://codeforces.com/api/"
REQUEST_SLEEP = 2.1          # IMPORTANT for CF API rate limit 
MAX_SUBS_PER_USER = None     # Set e.g. 4000 to speed up (optional)
OUT_DIR = "cf_dataset_ml"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------------
# Load Top15 topic vocabulary
# -----------------------------
top15_df = pd.read_csv("cf_dataset_v2_fixed/submissions_clean_top15.csv")
TOP15 = list(top15_df["topic"].value_counts().index)
TOP15_SET = set(TOP15)

# -----------------------------
# Load sampled users
# -----------------------------
USERS = pd.read_csv("cf_dataset_ml/users_3k.csv")["user_id"].dropna().astype(str).tolist()
print("Users loaded:", len(USERS))

# -----------------------------
# Helpers
# -----------------------------
def cf_get(endpoint, params=None, retries=6):
    if params is None:
        params = {}
    url = BASE + endpoint
    last = None

    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, timeout=30)
            data = r.json()
            if data.get("status") == "OK":
                return data["result"]
            last = data
            # backoff
            time.sleep(min(30, 2 + attempt * 3))
        except Exception as e:
            last = e
            time.sleep(min(30, 2 + attempt * 3))

    raise Exception(f"Codeforces API failed: {last}")

# -----------------------------
# Fetch problem metadata once
# -----------------------------
print("📥 Loading problem metadata (problemset.problems)...")
ps = cf_get("problemset.problems")  # returns dict with "problems" and "problemStatistics"
problems = ps["problems"]

prob_meta = {}
for p in problems:
    cid = p.get("contestId")
    idx = p.get("index")
    if cid is None or idx is None:
        continue
    prob_meta[(cid, idx)] = {
        "problem_name": p.get("name"),
        "tags": p.get("tags", []) or [],
        "problem_rating": p.get("rating", None)
    }

print("✅ Metadata loaded:", len(prob_meta))

# -----------------------------
# Build problem_attempt_history
# -----------------------------
rows = []
failures = []

for user in tqdm(USERS, desc="Step 1: building problem_attempt_history"):
    try:
        subs = cf_get("user.status", {"handle": user})
    except Exception as e:
        failures.append((user, str(e)))
        continue

    subs = sorted(subs, key=lambda x: x.get("creationTimeSeconds", 0))  # oldest->newest
    if MAX_SUBS_PER_USER:
        subs = subs[-MAX_SUBS_PER_USER:]

    per_problem = {}  # (contest_id, index) -> stats

    for s in subs:
        cid = s.get("contestId")
        prob = s.get("problem", {}) or {}
        idx = prob.get("index")
        verdict = s.get("verdict")
        ts = s.get("creationTimeSeconds")

        if cid is None or idx is None or ts is None:
            continue

        key = (cid, idx)
        meta = prob_meta.get(key, {
            "problem_name": prob.get("name", None),
            "tags": prob.get("tags", []) or [],
            "problem_rating": prob.get("rating", None),
        })

        tags = meta.get("tags", []) or []
        topics = [t for t in tags if t in TOP15_SET]
        if not topics:
            topics = ["other"] if "other" in TOP15_SET else []
        if not topics:
            continue

        if key not in per_problem:
            per_problem[key] = {
                "user_id": user,
                "contest_id": cid,
                "problem_index": idx,
                "problem_key": f"{cid}-{idx}",
                "problem_name": meta.get("problem_name"),
                "problem_rating": meta.get("problem_rating"),
                "topics": json.dumps(topics),  # JSON-safe
                "raw_tags": json.dumps(tags),  # JSON-safe
                "first_submission_time": ts,
                "first_ok_time": None,
                "total_submissions": 0,
                "submissions_until_ok": None,
                "solved": 0
            }

        st = per_problem[key]
        st["total_submissions"] += 1

        if verdict == "OK" and st["first_ok_time"] is None:
            st["first_ok_time"] = ts
            st["solved"] = 1
            st["submissions_until_ok"] = st["total_submissions"]

    for st in per_problem.values():
        if st["first_ok_time"] is not None:
            st["time_to_first_ac"] = st["first_ok_time"] - st["first_submission_time"]
        else:
            st["time_to_first_ac"] = None
        rows.append(st)

    time.sleep(REQUEST_SLEEP)

hist = pd.DataFrame(rows)

out_path = "cf_dataset_ml/problem_attempt_history.csv"
hist.to_csv(out_path, index=False)

print("\n✅ Saved:", out_path)
print("Total rows (user-problem):", len(hist))
print("Unique users in output:", hist["user_id"].nunique())
print("Example rows:")
print(hist.head(3))

if failures:
    fail_path = "cf_dataset_ml/failed_users_step1.csv"
    pd.DataFrame(failures, columns=["user_id", "error"]).to_csv(fail_path, index=False)
    print("\n⚠️ Some users failed. Saved:", fail_path, "| Count:", len(failures))


Users loaded: 3000
📥 Loading problem metadata (problemset.problems)...
✅ Metadata loaded: 10994


Step 1: building problem_attempt_history: 100%|██████████| 3000/3000 [2:50:37<00:00,  3.41s/it]   



✅ Saved: cf_dataset_ml/problem_attempt_history.csv
Total rows (user-problem): 370645
Unique users in output: 2989
Example rows:
        user_id  contest_id problem_index problem_key        problem_name  \
0  Ravinder2000           4             A         4-A          Watermelon   
1  Ravinder2000          71             A        71-A  Way Too Long Words   
2  Ravinder2000         118             A       118-A         String Task   

   problem_rating                         topics  \
0           800.0        ["brute force", "math"]   
1           800.0                    ["strings"]   
2          1000.0  ["implementation", "strings"]   

                        raw_tags  first_submission_time  first_ok_time  \
0        ["brute force", "math"]             1601488387   1.601488e+09   
1                    ["strings"]             1601488841   1.623349e+09   
2  ["implementation", "strings"]             1623351809   1.623352e+09   

   total_submissions  submissions_until_ok  solved  time

In [6]:
import pandas as pd
import numpy as np
import json

# -----------------------------
# Load TOP15 topic vocabulary
# -----------------------------
top15_df = pd.read_csv("cf_dataset_v2_fixed/submissions_clean_top15.csv")
TOP15 = list(top15_df["topic"].value_counts().index)

# -----------------------------
# Load problem-level history
# -----------------------------
hist = pd.read_csv("cf_dataset_ml/problem_attempt_history.csv")

# Parse topics JSON string -> list
hist["topics"] = hist["topics"].apply(lambda x: json.loads(x) if isinstance(x, str) else [])

all_rows = []

for user_id, g in hist.groupby("user_id"):
    # multi-label explode
    g_exp = g.explode("topics").rename(columns={"topics": "topic"})
    g_exp = g_exp[g_exp["topic"].isin(TOP15)]

    attempted = g_exp.groupby("topic").size()
    solved = g_exp[g_exp["solved"] == 1].groupby("topic").size()
    submissions = g_exp.groupby("topic")["total_submissions"].sum()

    # median time to first AC among solved problems
    tta = (
        g_exp[g_exp["time_to_first_ac"].notna()]
        .groupby("topic")["time_to_first_ac"]
        .median()
    )

    # include attempt=0 topics too
    for t in TOP15:
        a = int(attempted.get(t, 0))
        s = int(solved.get(t, 0))
        sub = int(submissions.get(t, 0))
        acc = (s / a) if a > 0 else 0.0
        struggle = (sub / a) if a > 0 else 0.0
        med_tta = float(tta.get(t, np.nan)) if t in tta.index else np.nan

        all_rows.append({
            "user_id": user_id,
            "topic": t,
            "attempted_unique": a,
            "solved_unique": s,
            "submissions": sub,
            "accuracy_unique": acc,
            "struggle_score": struggle,
            "median_time_to_first_ac": med_tta
        })

feat = pd.DataFrame(all_rows)

# -----------------------------
# Topic state labeling (robust)
# - avoid "1/1 = STRONG" issue by requiring min attempts
# -----------------------------
def label_topic_state(row, med_attempts):
    a = row["attempted_unique"]
    acc = row["accuracy_unique"]

    if a == 0:
        return "MEDIUM"
    if a >= max(5, med_attempts) and acc < 0.45:
        return "WEAK"
    if a >= 5 and acc >= 0.70:
        return "STRONG"
    return "MEDIUM"

feat["topic_state"] = "MEDIUM"
for user_id, g in feat.groupby("user_id"):
    med_attempts = int(g["attempted_unique"].median())
    feat.loc[g.index, "topic_state"] = g.apply(lambda r: label_topic_state(r, med_attempts), axis=1)

# Save
out_path = "cf_dataset_ml/user_topic_features_ml.csv"
feat.to_csv(out_path, index=False)

print("✅ Saved:", out_path)
print("Rows:", len(feat), "| Users:", feat["user_id"].nunique(), "| Topics per user:", feat["topic"].nunique())
print(feat.head(10))


✅ Saved: cf_dataset_ml/user_topic_features_ml.csv
Rows: 44835 | Users: 2989 | Topics per user: 15
      user_id                    topic  attempted_unique  solved_unique  \
0  -BALERION-                     math                20             15   
1  -BALERION-                   greedy                20             17   
2  -BALERION-           implementation                11             10   
3  -BALERION-                       dp                 4              1   
4  -BALERION-          data structures                 1              1   
5  -BALERION-                    other                 3              3   
6  -BALERION-              brute force                 5              5   
7  -BALERION-  constructive algorithms                10              7   
8  -BALERION-                   graphs                 1              0   
9  -BALERION-            binary search                 2              1   

   submissions  accuracy_unique  struggle_score  median_time_to_first_ac  \


In [7]:
import pandas as pd
import json
from datetime import datetime, UTC

# -----------------------------
# Load TOP15
# -----------------------------
top15_df = pd.read_csv("cf_dataset_v2_fixed/submissions_clean_top15.csv")
TOP15 = list(top15_df["topic"].value_counts().index)

# -----------------------------
# Load problem-level history
# -----------------------------
hist = pd.read_csv("cf_dataset_ml/problem_attempt_history.csv")
hist["topics"] = hist["topics"].apply(lambda x: json.loads(x) if isinstance(x, str) else [])

# -----------------------------
# year_week helper
# -----------------------------
def year_week_from_ts(ts):
    dt = datetime.fromtimestamp(int(ts), UTC)  # timezone-aware UTC
    y, w, _ = dt.isocalendar()
    return f"{y}-W{w:02d}"

hist["year_week"] = hist["first_submission_time"].apply(year_week_from_ts)

# -----------------------------
# Build weekly cumulative accuracy snapshots
# -----------------------------
rows = []

for user_id, g in hist.groupby("user_id"):
    g = g.sort_values("first_submission_time")

    # multi-label explode
    g_exp = g.explode("topics").rename(columns={"topics": "topic"})
    g_exp = g_exp[g_exp["topic"].isin(TOP15)]

    attempted = {t: set() for t in TOP15}
    solved = {t: set() for t in TOP15}

    weekly_records = {}  # year_week -> {topic: acc}

    for _, row in g_exp.iterrows():
        t = row["topic"]
        prob_key = (int(row["contest_id"]), str(row["problem_index"]))
        yw = row["year_week"]

        attempted[t].add(prob_key)
        if int(row["solved"]) == 1:
            solved[t].add(prob_key)

        if yw not in weekly_records:
            weekly_records[yw] = {}

        # snapshot for ALL topics at this week
        for topic in TOP15:
            a = len(attempted[topic])
            s = len(solved[topic])
            weekly_records[yw][topic] = (s / a) if a > 0 else 0.0

    # convert to rows
    for yw, topic_accs in weekly_records.items():
        rec = {"user_id": user_id, "year_week": yw}
        rec.update(topic_accs)
        rows.append(rec)

trend = pd.DataFrame(rows)

# -----------------------------
# Sort year_week correctly
# -----------------------------
def parse_yw(s):
    y, w = s.split("-W")
    return int(y), int(w)

trend[["yw_year", "yw_week"]] = trend["year_week"].apply(lambda x: pd.Series(parse_yw(x)))
trend = trend.sort_values(["user_id", "yw_year", "yw_week"]).reset_index(drop=True)

# -----------------------------
# Forward fill per user
# -----------------------------
trend[TOP15] = trend.groupby("user_id")[TOP15].ffill().fillna(0.0)

trend = trend.drop(columns=["yw_year", "yw_week"])

# Save
out_path = "cf_dataset_ml/user_topic_weekly_trend_unique.csv"
trend.to_csv(out_path, index=False)

print("✅ Saved:", out_path)
print("Rows:", len(trend), "| Users:", trend["user_id"].nunique())
print(trend.head(5))


✅ Saved: cf_dataset_ml/user_topic_weekly_trend_unique.csv
Rows: 60244 | Users: 2989
      user_id year_week      math    greedy  implementation   dp  \
0  -BALERION-  2025-W50  1.000000  1.000000        1.000000  0.0   
1  -BALERION-  2025-W51  1.000000  0.857143        1.000000  0.0   
2  -BALERION-  2025-W52  0.875000  0.777778        1.000000  0.5   
3  -BALERION-  2026-W01  0.875000  0.800000        1.000000  0.5   
4  -BALERION-  2026-W02  0.833333  0.812500        0.888889  0.5   

   data structures  other  brute force  constructive algorithms  graphs  \
0              0.0    0.0          1.0                 1.000000     0.0   
1              0.0    0.0          1.0                 0.500000     0.0   
2              0.0    0.0          1.0                 0.333333     0.0   
3              0.0    0.0          1.0                 0.500000     0.0   
4              0.0    1.0          1.0                 0.571429     0.0   

   binary search  sortings  trees  number theory   strin

In [9]:
import time
import requests
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import joblib

# -----------------------------
# Config
# -----------------------------
BASE = "https://codeforces.com/api/"
REQUEST_SLEEP = 2.1  # CF limit 

# -----------------------------
# Load TOP15
# -----------------------------
top15_df = pd.read_csv("cf_dataset_v2_fixed/submissions_clean_top15.csv")
TOP15 = list(top15_df["topic"].value_counts().index)

# -----------------------------
# Load features + trend
# -----------------------------
feat = pd.read_csv("cf_dataset_ml/user_topic_features_ml.csv")
trend = pd.read_csv("cf_dataset_ml/user_topic_weekly_trend_unique.csv")

# Users list (same 3k)
users = pd.read_csv("cf_dataset_ml/users_3k.csv")["user_id"].dropna().astype(str).tolist()
print("Users:", len(users))

# -----------------------------
# CF helper
# -----------------------------
def cf_get(endpoint, params=None, retries=6):
    if params is None: params = {}
    url = BASE + endpoint
    last = None
    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, timeout=30)
            data = r.json()
            if data.get("status") == "OK":
                return data["result"]
            last = data
            time.sleep(min(30, 2 + attempt * 3))
        except Exception as e:
            last = e
            time.sleep(min(30, 2 + attempt * 3))
    raise Exception(f"API failed: {last}")

# -----------------------------
# Fetch user profiles (rating) in chunks
# -----------------------------
profiles = []
chunk_size = 400  # safe URL length
for i in range(0, len(users), chunk_size):
    chunk = users[i:i+chunk_size]
    res = cf_get("user.info", {"handles": ";".join(chunk)})

    for u in res:
        profiles.append({
            "user_id": u.get("handle"),
            "rating": u.get("rating", 0),
            "max_rating": u.get("maxRating", 0),
            "rank": u.get("rank", ""),
            "max_rank": u.get("maxRank", "")
        })

    time.sleep(REQUEST_SLEEP)

profile = pd.DataFrame(profiles)
profile.to_csv("cf_dataset_ml/user_profile.csv", index=False)
print("✅ Saved cf_dataset_ml/user_profile.csv | Rows:", len(profile))

# -----------------------------
# Trend slope per user
# -----------------------------
def compute_trend_slope(g):
    topic_cols = [c for c in g.columns if c not in ["user_id", "year_week"]]
    g = g.sort_values("year_week")
    y = g[topic_cols].mean(axis=1).values
    if len(y) < 2:
        return 0.0
    x = np.arange(len(y))
    return np.polyfit(x, y, 1)[0]

trend_slope = (
    trend.groupby("user_id")
    .apply(compute_trend_slope)
    .reset_index(name="trend_slope")
)

# -----------------------------
# Pivot topic features wide
# -----------------------------
def pivot_feature(df, colname):
    p = df.pivot(index="user_id", columns="topic", values=colname)
    p.columns = [f"{colname}_{c}" for c in p.columns]
    return p

acc_pivot = pivot_feature(feat, "accuracy_unique")
cov_pivot = pivot_feature(feat, "attempted_unique")
str_pivot = pivot_feature(feat, "struggle_score")

X = acc_pivot.join(cov_pivot).join(str_pivot).reset_index()

# Add rating + trend_slope
X = X.merge(profile[["user_id", "rating"]], on="user_id", how="left")
X = X.merge(trend_slope, on="user_id", how="left")

X = X.fillna(0)

user_ids = X["user_id"]
X_vals = X.drop(columns=["user_id"])

# -----------------------------
# Save feature column order for inference
# -----------------------------
with open("cf_dataset_ml/training_feature_columns_v2.txt", "w") as f:
    for c in X_vals.columns:
        f.write(c + "\n")
print("✅ Saved cf_dataset_ml/training_feature_columns_v2.txt")

# -----------------------------
# Scale + choose best K
# -----------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_vals)

best_k, best_score = 2, -1
for k in range(2, 8):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    print(f"K={k}, Silhouette={score:.4f}")
    if score > best_score:
        best_score, best_k = score, k

print("✅ Best K:", best_k)

# -----------------------------
# Final clustering
# -----------------------------
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

cluster_df = pd.DataFrame({
    "user_id": user_ids,
    "cluster": [f"C{c}" for c in clusters]
})
cluster_df.to_csv("cf_dataset_ml/user_clusters_v2.csv", index=False)
print("✅ Saved cf_dataset_ml/user_clusters_v2.csv")

# -----------------------------
# Save artifacts
# -----------------------------
joblib.dump(scaler, "cf_dataset_ml/scaler_v2.pkl")
joblib.dump(kmeans, "cf_dataset_ml/kmeans_v2.pkl")
print("✅ Saved cf_dataset_ml/scaler_v2.pkl and cf_dataset_ml/kmeans_v2.pkl")


Users: 3000
✅ Saved cf_dataset_ml/user_profile.csv | Rows: 3000
✅ Saved cf_dataset_ml/training_feature_columns_v2.txt
K=2, Silhouette=0.2234
K=3, Silhouette=0.2210
K=4, Silhouette=0.1656
K=5, Silhouette=0.1651
K=6, Silhouette=0.1524
K=7, Silhouette=0.1548
✅ Best K: 2
✅ Saved cf_dataset_ml/user_clusters_v2.csv
✅ Saved cf_dataset_ml/scaler_v2.pkl and cf_dataset_ml/kmeans_v2.pkl


In [10]:
import pandas as pd
import json
import numpy as np

# -----------------------------
# Load problem history + clusters
# -----------------------------
hist = pd.read_csv("cf_dataset_ml/problem_attempt_history.csv")
hist["topics"] = hist["topics"].apply(lambda x: json.loads(x) if isinstance(x, str) else [])

clusters = pd.read_csv("cf_dataset_ml/user_clusters_v2.csv")

# attach cluster to each user-problem row
hist = hist.merge(clusters, on="user_id", how="left")
hist = hist[hist["cluster"].notna()].copy()

# explode topics (multi-label)
df = hist.explode("topics").rename(columns={"topics": "topic"})
df["problem_key"] = df["contest_id"].astype(str) + "-" + df["problem_index"].astype(str)

# -----------------------------
# Compute stats
# -----------------------------
attempt_users = df.groupby(["cluster","topic","problem_key"])["user_id"].nunique().rename("attempt_users")

solve_users = (
    df[df["solved"] == 1]
    .groupby(["cluster","topic","problem_key"])["user_id"]
    .nunique()
    .rename("solve_users")
)

med_sub_until_ok = (
    df[df["solved"] == 1]
    .groupby(["cluster","topic","problem_key"])["submissions_until_ok"]
    .median()
    .rename("median_submissions_until_ok")
)

med_time_to_ac = (
    df[df["solved"] == 1]
    .groupby(["cluster","topic","problem_key"])["time_to_first_ac"]
    .median()
    .rename("median_time_to_first_ac")
)

meta = (
    df.groupby(["cluster","topic","problem_key"], as_index=False)
    .agg(
        contest_id=("contest_id","first"),
        problem_index=("problem_index","first"),
        problem_name=("problem_name","first"),
        problem_rating=("problem_rating","first")
    )
    .set_index(["cluster","topic","problem_key"])
)

stats = meta.join(attempt_users).join(solve_users).join(med_sub_until_ok).join(med_time_to_ac).reset_index()

stats["attempt_users"] = stats["attempt_users"].fillna(0).astype(int)
stats["solve_users"] = stats["solve_users"].fillna(0).astype(int)

stats["success_rate"] = stats.apply(
    lambda r: (r["solve_users"] / r["attempt_users"]) if r["attempt_users"] > 0 else 0.0,
    axis=1
)

# Add problem link
stats["cf_link"] = stats.apply(
    lambda r: f"https://codeforces.com/contest/{int(r['contest_id'])}/problem/{r['problem_index']}",
    axis=1
)

# Save
out_path = "cf_dataset_ml/cluster_topic_problem_stats.csv"
stats.to_csv(out_path, index=False)

print("✅ Saved:", out_path)
print("Rows:", len(stats), "| Clusters:", stats["cluster"].nunique(), "| Topics:", stats["topic"].nunique())
print(stats.head(5))


✅ Saved: cf_dataset_ml/cluster_topic_problem_stats.csv
Rows: 48647 | Clusters: 2 | Topics: 15
  cluster          topic problem_key  contest_id problem_index  \
0      C0  binary search      1004-E        1004             E   
1      C0  binary search      1006-C        1006             C   
2      C0  binary search      1007-C        1007             C   
3      C0  binary search      1008-E        1008             E   
4      C0  binary search       101-B         101             B   

               problem_name  problem_rating  attempt_users  solve_users  \
0       Sonya and Ice Cream          2400.0              3            1   
1  Three Parts of the Array          1200.0             66           63   
2         Guess two numbers          3000.0              3            3   
3         Guess two numbers          3000.0              2            1   
4                     Buses          1700.0              7            6   

   median_submissions_until_ok  median_time_to_first_ac  s

In [11]:
import joblib
import requests
import time
import json
import pandas as pd
import numpy as np
from datetime import datetime, UTC

# -----------------------------
# Config
# -----------------------------
BASE = "https://codeforces.com/api/"
REQUEST_SLEEP = 2.1  # CF limit 
MAX_SUBS_PER_USER = None  # optional cap for speed

# -----------------------------
# Load TOP15
# -----------------------------
top15_df = pd.read_csv("cf_dataset_v2_fixed/submissions_clean_top15.csv")
TOP15 = list(top15_df["topic"].value_counts().index)
TOP15_SET = set(TOP15)

# -----------------------------
# Load artifacts
# -----------------------------
scaler = joblib.load("cf_dataset_ml/scaler_v2.pkl")
kmeans = joblib.load("cf_dataset_ml/kmeans_v2.pkl")
stats  = pd.read_csv("cf_dataset_ml/cluster_topic_problem_stats.csv")

with open("cf_dataset_ml/training_feature_columns_v2.txt", "r") as f:
    FEATURE_COLS = [line.strip() for line in f if line.strip()]

# -----------------------------
# CF helper
# -----------------------------
def cf_get(endpoint, params=None, retries=6):
    if params is None: params = {}
    url = BASE + endpoint
    last = None
    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, timeout=30)
            data = r.json()
            if data.get("status") == "OK":
                return data["result"]
            last = data
            time.sleep(min(30, 2 + attempt * 3))
        except Exception as e:
            last = e
            time.sleep(min(30, 2 + attempt * 3))
    raise Exception(f"API failed: {last}")

def align_columns(vec_df):
    for c in FEATURE_COLS:
        if c not in vec_df.columns:
            vec_df[c] = 0.0
    return vec_df[FEATURE_COLS]

def year_week_from_ts(ts):
    dt = datetime.fromtimestamp(int(ts), UTC)
    y, w, _ = dt.isocalendar()
    return f"{y}-W{w:02d}"

# -----------------------------
# Load problem metadata once
# -----------------------------
print("📥 Loading problem metadata (problemset.problems)...")
ps = cf_get("problemset.problems")
prob_meta = {(p["contestId"], p["index"]): {
    "problem_name": p.get("name"),
    "tags": p.get("tags", []) or [],
    "problem_rating": p.get("rating", None)
} for p in ps["problems"]}
print("✅ Metadata loaded:", len(prob_meta))

# -----------------------------
# A) Platform tag counts (unique solved problems)
# -----------------------------
def print_codeforces_platform_tag_counts(handle, top_k=30):
    subs = cf_get("user.status", {"handle": handle})

    solved_problems = set()
    tag_counter = {}

    for s in subs:
        if s.get("verdict") != "OK":
            continue
        prob = s.get("problem", {}) or {}
        cid = prob.get("contestId")
        idx = prob.get("index")
        tags = prob.get("tags", []) or []
        if cid is None or idx is None:
            continue

        key = (cid, idx)
        if key in solved_problems:
            continue
        solved_problems.add(key)

        for t in tags:
            tag_counter[t] = tag_counter.get(t, 0) + 1

    sorted_tags = sorted(tag_counter.items(), key=lambda x: x[1], reverse=True)

    print("\n--- Codeforces Platform Tag Counts (unique solved problems) ---")
    for t, c in sorted_tags[:top_k]:
        print(f"{t:25s} : {c}")

# -----------------------------
# B) Build new user's problem history (problem-level, multi-label TOP15)
# -----------------------------
def build_new_user_problem_history(handle, limit=MAX_SUBS_PER_USER):
    subs = cf_get("user.status", {"handle": handle})
    subs = sorted(subs, key=lambda x: x.get("creationTimeSeconds", 0))  # oldest->newest
    if limit:
        subs = subs[-limit:]

    per_problem = {}
    for s in subs:
        cid = s.get("contestId")
        prob = s.get("problem", {}) or {}
        idx = prob.get("index")
        verdict = s.get("verdict")
        ts = s.get("creationTimeSeconds")

        if cid is None or idx is None or ts is None:
            continue

        key = (cid, idx)
        meta = prob_meta.get(key, {
            "problem_name": prob.get("name"),
            "tags": prob.get("tags", []) or [],
            "problem_rating": prob.get("rating", None)
        })

        tags = meta.get("tags", []) or []
        topics = [t for t in tags if t in TOP15_SET]
        if not topics:
            topics = ["other"] if "other" in TOP15_SET else []
        if not topics:
            continue

        if key not in per_problem:
            per_problem[key] = {
                "contest_id": cid,
                "problem_index": idx,
                "problem_key": f"{cid}-{idx}",
                "problem_name": meta.get("problem_name"),
                "problem_rating": meta.get("problem_rating"),
                "topics": topics,
                "first_submission_time": ts,
                "first_ok_time": None,
                "total_submissions": 0,
                "submissions_until_ok": None,
                "solved": 0,
                "year_week": year_week_from_ts(ts)
            }

        st = per_problem[key]
        st["total_submissions"] += 1

        if verdict == "OK" and st["first_ok_time"] is None:
            st["first_ok_time"] = ts
            st["solved"] = 1
            st["submissions_until_ok"] = st["total_submissions"]

    for st in per_problem.values():
        st["time_to_first_ac"] = (st["first_ok_time"] - st["first_submission_time"]) if st["first_ok_time"] else np.nan

    return pd.DataFrame(per_problem.values())

# -----------------------------
# C) Build topic features table weak->strong (includes attempt=0)
# -----------------------------
def build_user_topic_features_from_history(problem_hist_df):
    g_exp = problem_hist_df.explode("topics").rename(columns={"topics":"topic"})
    g_exp = g_exp[g_exp["topic"].isin(TOP15)]

    attempted = g_exp.groupby("topic").size()
    solved = g_exp[g_exp["solved"]==1].groupby("topic").size()
    submissions = g_exp.groupby("topic")["total_submissions"].sum()

    rows=[]
    for t in TOP15:
        a = int(attempted.get(t, 0))
        s = int(solved.get(t, 0))
        sub = int(submissions.get(t, 0))
        acc = (s/a) if a>0 else 0.0
        struggle = (sub/a) if a>0 else 0.0
        rows.append({
            "topic": t,
            "attempted_unique": a,
            "solved_unique": s,
            "accuracy_unique": acc,
            "struggle_score": struggle
        })
    out = pd.DataFrame(rows)

    med_attempts = int(out["attempted_unique"].median())
    def label(r):
        a=r["attempted_unique"]; acc=r["accuracy_unique"]
        if a==0: return "MEDIUM"
        if a>=max(5, med_attempts) and acc<0.45: return "WEAK"
        if a>=5 and acc>=0.70: return "STRONG"
        return "MEDIUM"
    out["topic_state"] = out.apply(label, axis=1)

    # weak->strong
    return out.sort_values(["accuracy_unique","attempted_unique"], ascending=[True, True]).reset_index(drop=True)

# -----------------------------
# D) Trend slope from history (simple weekly accuracy series)
# -----------------------------
def compute_trend_slope_from_history(problem_hist_df):
    g_exp = problem_hist_df.explode("topics").rename(columns={"topics":"topic"})
    g_exp = g_exp[g_exp["topic"].isin(TOP15)]

    weekly = (
        g_exp.groupby(["year_week","topic"], as_index=False)
        .agg(attempted=("problem_key","count"), solved=("solved","sum"))
    )
    weekly["acc"] = weekly["solved"] / weekly["attempted"]

    overall = weekly.groupby("year_week")["acc"].mean().reset_index().sort_values("year_week")
    if len(overall) < 2:
        return 0.0
    y = overall["acc"].values
    x = np.arange(len(y))
    return float(np.polyfit(x, y, 1)[0])

# -----------------------------
# E) Build cluster vector (must match Step 4 schema)
# -----------------------------
def build_cluster_vector(rating, topic_tbl, trend_slope):
    row = {"rating": float(rating), "trend_slope": float(trend_slope)}
    for _, r in topic_tbl.iterrows():
        t = r["topic"]
        row[f"accuracy_unique_{t}"] = float(r["accuracy_unique"])
        row[f"attempted_unique_{t}"] = float(r["attempted_unique"])
        row[f"struggle_score_{t}"] = float(r["struggle_score"])
    return pd.DataFrame([row])

# -----------------------------
# F) Recommend for one topic (cluster + topic)
# -----------------------------
def recommend_for_cluster_topic(cluster_label, topic, solved_keys, top_n=5, user_rating=None, rating_band=300):
    cand = stats[(stats["cluster"]==cluster_label) & (stats["topic"]==topic)].copy()
    cand = cand[~cand["problem_key"].isin(solved_keys)]

    # rating band filter (OA-friendly)
    if user_rating is not None and "problem_rating" in cand.columns:
        low, high = user_rating - rating_band, user_rating + rating_band
        cand2 = cand[(cand["problem_rating"].notna()) &
                     (cand["problem_rating"]>=low) &
                     (cand["problem_rating"]<=high)].copy()
        if not cand2.empty:
            cand = cand2

    if cand.empty:
        return cand

    cand["median_submissions_until_ok"] = cand["median_submissions_until_ok"].fillna(999)
    cand["median_time_to_first_ac"] = cand["median_time_to_first_ac"].fillna(10**12)

    cand = cand.sort_values(
        ["success_rate","solve_users","median_submissions_until_ok","median_time_to_first_ac"],
        ascending=[False, False, True, True]
    ).head(top_n)

    return cand[["contest_id","problem_index","problem_name","problem_rating","success_rate","solve_users","attempt_users","cf_link"]]

# -----------------------------
# G) Full report function
# -----------------------------
def new_user_report(handle, top_n=5):
    # platform counts
    print_codeforces_platform_tag_counts(handle, top_k=30)

    # rating
    info = cf_get("user.info", {"handles": handle})[0]
    rating = info.get("rating", None)
    if rating is None:
        raise ValueError("User has no rating yet (unrated).")

    # build history + features
    hist_df = build_new_user_problem_history(handle)
    topic_tbl = build_user_topic_features_from_history(hist_df)
    trend_slope = compute_trend_slope_from_history(hist_df)

    # cluster prediction
    vec = build_cluster_vector(rating, topic_tbl, trend_slope)
    vec_aligned = align_columns(vec)

    cluster_id = int(kmeans.predict(scaler.transform(vec_aligned))[0])
    cluster = f"C{cluster_id}"

    # solved filter
    solved_keys = set(hist_df[hist_df["solved"]==1]["problem_key"])

    print(f"\nUser: {handle} Rating: {int(rating)} Predicted cluster: {cluster}\n")

    print("Topics weak -> strong:")
    print(
        topic_tbl[["topic","topic_state","accuracy_unique","attempted_unique"]]
        .rename(columns={"accuracy_unique":"accuracy","attempted_unique":"attempts"})
        .to_string(index=False)
    )

    weakest_topic = topic_tbl.iloc[0]["topic"]
    print(f"\nWeakest topic: {weakest_topic}")

    recs = recommend_for_cluster_topic(cluster, weakest_topic, solved_keys, top_n=top_n, user_rating=rating)
    if recs.empty:
        print("No recommendations found (all solved / no candidates).")
    else:
        print(recs.to_string(index=False))

# Example:
# new_user_report("mohd_fariq", top_n=5)


📥 Loading problem metadata (problemset.problems)...
✅ Metadata loaded: 10993


In [12]:
new_user_report("mohd_fariq", top_n=5)


--- Codeforces Platform Tag Counts (unique solved problems) ---
greedy                    : 141
math                      : 101
implementation            : 92
brute force               : 56
constructive algorithms   : 54
sortings                  : 42
strings                   : 27
dp                        : 23
binary search             : 22
two pointers              : 22
number theory             : 16
data structures           : 15
games                     : 10
bitmasks                  : 7
graphs                    : 4
combinatorics             : 3
shortest paths            : 2
trees                     : 2
dfs and similar           : 2
geometry                  : 2
dsu                       : 1
matrices                  : 1
ternary search            : 1

User: mohd_fariq Rating: 1163 Predicted cluster: C0

Topics weak -> strong:
                  topic topic_state  accuracy  attempts
                 graphs      STRONG  0.800000         5
               sortings      STRONG  0.82